## Consumption of Pornographic Content by Age and Education

In [1]:
import pandas as pd
import gc

### Load Data

The 2004 comScore data are already grouped by machine_id and domain_name. The data has four columns: 
a. machine_id, 
b. domain_name, 
c. total duration of visits to the site, 
d. number of visits

Each row gives the total visits to and total duration spent on a domain.

We merge this with data from pydomains and the Trusted Source API. We then calculate the time spent on and total visits to pornographic domains based on a variety of measures. We also create two other columns that track proportion of visits, and proportion of time.

In [2]:
YEAR = 2004

In [3]:
# Read in data
idf = pd.read_csv('/opt/comscore/pydomains/app2/cs%04d_grp_machine_domain.csv.bz2' % YEAR)

In [4]:
# Load the pydomains data
pydom_df = pd.read_csv('/opt/comscore/pydomains/cs%04d_unique_domains_pydomains.csv.bz2' % YEAR, usecols=['domain_names', 'shalla_2017_cat', 'pred_shalla_2017_lab', 'pred_shalla_2017_prob_porn', 'pred_toulouse_2017_prob_adult'], encoding='latin1')
# rename column
pydom_df.rename(columns={'domain_names': 'domain_name'}, inplace=True)

In [5]:
# Load trusted data
tdf = pd.read_csv('/opt/comscore/pydomains/comScore_unique_2004.csv')
# rename column
tdf.rename(columns={'unique_url': 'domain_name', 'url_class': 'trusted_cat'}, inplace=True)

### Left join Trusted Source and PyDomains

In [6]:
# Left join with pydomain
pdf = idf.merge(pydom_df, how='left', on='domain_name')

# Left join with Trusted
pdf = pdf.merge(tdf, how='left', on='domain_name')

For domains that are already in the labeled datasets, we use the labels from there.

In [7]:
# -1 for Unknown, 0 - No, 1 - Yes
pdf['shalla_trusted_porn'] = 0
pdf.loc[pdf.shalla_2017_cat.isnull() & (pdf.trusted_cat.isnull() | (pdf.trusted_cat == 'UNKNOWN')), 'shalla_trusted_porn'] = -1
pdf.loc[pdf.shalla_2017_cat.notnull() & pdf.shalla_2017_cat.str.contains('porn', case=False), 'shalla_trusted_porn'] = 1
pdf.loc[pdf.trusted_cat.notnull() & pdf.trusted_cat.str.contains('porn', case=False), 'shalla_trusted_porn'] = 1

pdf[['shalla_2017_cat', 'trusted_cat', 'shalla_trusted_porn']].head()

,shalla_2017_cat,trusted_cat,shalla_trusted_porn
0,NaN,UNKNOWN,-1
1,porn|hobby/games-online,Games,1
2,NaN,Internet Services,0
3,NaN,Online Shopping,0
4,porn,Pornography,1


### Unique domain names

In [8]:
udf = pdf.drop_duplicates(subset='domain_name').copy()
# Total unique domains
len(udf)

1011145

### Total number of domains for which the label must be imputed

Curated lists generally only information about the kind of content hosted by a small fraction of domains. Commercial APIs generally are a lot better but still miss a sizable chunk. In Shallist, for instance, only about 22% of the domains in the data have category assigned to them (see below). For Trusted Source, the commensurate number is nearly 86%. In all, we know the category of about 86% of the domains.

In [9]:
# -1 for Unknown, 0 - No, 1 - Yes
udf['shalla_cat_porn'] = -1
udf.loc[udf.shalla_2017_cat.notnull() & (udf.shalla_2017_cat.str.contains('unknown', case=False) == False), 'shalla_cat_porn'] = 0
udf.loc[udf.shalla_2017_cat.notnull() & udf.shalla_2017_cat.str.contains('porn', case=False), 'shalla_cat_porn'] = 1
udf.groupby('shalla_cat_porn').agg({'domain_name': 'count'})/udf.shape[0]

,domain_name
shalla_cat_porn,
-1,0.780723
0,0.096061
1,0.123216


In [10]:
# -1 for Unknown, 0 - No, 1 - Yes
udf['trusted_cat_porn'] = -1
udf.loc[udf.trusted_cat.notnull() & (udf.trusted_cat.str.contains('unknown', case=False) == False), 'trusted_cat_porn'] = 0
udf.loc[udf.trusted_cat.notnull() & udf.trusted_cat.str.contains('porn', case=False), 'trusted_cat_porn'] = 1
udf.groupby('trusted_cat_porn').agg({'domain_name': 'count'})/udf.shape[0]

,domain_name
trusted_cat_porn,
-1,0.156131
0,0.691121
1,0.152748


In [11]:
udf.groupby('shalla_trusted_porn').agg({'domain_name': 'count'})/udf.shape[0]

,domain_name
shalla_trusted_porn,
-1,0.154937
0,0.674917
1,0.170146


### Impact of Different Cut-offs

Next, we use the labeled data (from Trusted and Shallalist) to pick different probability cut-offs to test how inferences changes. We choose three: one that minimizes FP+FN, one that gives us far fewer FP, and one that gives us far fewer FN. (We cast a wide net.)

To get the value that minimizes FP+FN for a particular category in a multi-class prediction problem, we [run an optimization algorithm](https://github.com/soodoku/optimal_softmax_cutoffs).

In [12]:
# Filter out unknown (-1)
odf = udf[udf.shalla_trusted_porn != -1][['shalla_trusted_porn', 'pred_shalla_2017_prob_porn', 'pred_toulouse_2017_prob_adult']].copy()
odf.head()

,shalla_trusted_porn,pred_shalla_2017_prob_porn,pred_toulouse_2017_prob_adult
1,1,0.240971,0.341799
2,0,0.854591,0.996538
3,0,0.109746,0.115253
4,1,0.979598,0.985338
5,0,0.194238,0.406939


In [13]:
prob_shalla = {}
# the prob. threshold to get mininum FN+FP
prob_shalla['prob_shalla_min_fn_fp'] = 0.91

In [14]:
# FIXME: manual try to predict with reduce FN and FP
prob_shalla['prob_shalla_reduce_fn'] = 0.5
prob_shalla['prob_shalla_reduce_fp'] = 0.99

In [15]:
prob_toulouse = {}
# the prob. threshold to get mininum FN+FP
prob_toulouse['prob_toulouse_min_fn_fp'] = 0.91

In [16]:
# We try different probability cutoffs to show how inferences change based on trading false positives for false negatives.
# Shalla model
for c in prob_shalla:
    prob = prob_shalla[c]
    print(c, prob)
    pdf.loc[pdf.pred_shalla_2017_prob_porn <= prob, c] = False
    pdf.loc[pdf.pred_shalla_2017_prob_porn > prob, c] = True
    pdf.loc[pdf.shalla_trusted_porn == 1, c] = True
    pdf.loc[pdf.shalla_trusted_porn == 0, c] = False

prob_shalla_reduce_fp 0.99
prob_shalla_min_fn_fp 0.91
prob_shalla_reduce_fn 0.5


In [17]:
# Toulose model
for c in prob_toulouse:
    prob = prob_toulouse[c]
    print(c, prob)
    pdf.loc[pdf.pred_toulouse_2017_prob_adult <= prob, c] = False
    pdf.loc[pdf.pred_toulouse_2017_prob_adult > prob, c] = True
    pdf.loc[pdf.shalla_trusted_porn == 1, c] = True
    pdf.loc[pdf.shalla_trusted_porn == 0, c] = False

prob_toulouse_min_fn_fp 0.91


In [18]:
agg_sum = {'total_time': sum, 'total_visits': sum}
for c in prob_shalla:
    pdf.loc[pdf[c], 'total_time_porn_{}'.format(c)] = pdf['total_time']
    pdf.loc[pdf[c] == False, 'total_time_porn_{}'.format(c)] = 0
    pdf.loc[pdf[c], 'total_visits_porn_{}'.format(c)] = pdf['total_visits']
    pdf.loc[pdf[c] == False, 'total_visits_porn_{}'.format(c)] = 0
    agg_sum['total_time_porn_{}'.format(c)] = sum
    agg_sum['total_visits_porn_{}'.format(c)] = sum

for c in prob_toulouse:
    pdf.loc[pdf[c], 'total_time_porn_{}'.format(c)] = pdf['total_time']
    pdf.loc[pdf[c] == False, 'total_time_porn_{}'.format(c)] = 0
    pdf.loc[pdf[c], 'total_visits_porn_{}'.format(c)] = pdf['total_visits']
    pdf.loc[pdf[c] == False, 'total_visits_porn_{}'.format(c)] = 0
    agg_sum['total_time_porn_{}'.format(c)] = sum
    agg_sum['total_visits_porn_{}'.format(c)] = sum

In [19]:
# agg. total_time and total_visits by machine_id
gdf = pdf.groupby(['machine_id']).agg(agg_sum)

### We join the data with demographic data at the household level, recoding demographic codes to semantic labels.

In [20]:
# Load household level demographics data
dem_df = pd.read_csv('/opt/comscore/demographics_by_machine_id/demographics%d.csv' % YEAR, usecols = ['machine_id', 'hoh_oldest_age', 'hoh_most_education'])

dem_df['hoh_oldest_age'] = dem_df['hoh_oldest_age'].replace({1: '18-20', 
                                                             2: '21-24',
                                                             3: '25-29', 
                                                             4: '30-34',
                                                             5: '35-39',
                                                             6: '40-44',
                                                             7: '45-49',
                                                             8: '50-54',
                                                             9: '55-59',
                                                             10: '60-64',
                                                             11: '65 and over',
                                                             99: 'Missing'})

# FIXME: replace '**' to 99
dem_df['hoh_most_education'] = dem_df['hoh_most_education'].astype(str).replace({
                                                                 '**': 99})

edu = {0: 'Less than a high school diploma',
           1: 'High school diploma or equivalent', 
           2: 'Some college but no degree', 
           3: 'Associate degree', 
           4: 'Bachelor’s degree',
           5: 'Graduate degree',
           99: 'Missing'}

dem_df['hoh_most_education'] = dem_df['hoh_most_education'].astype(int).replace(edu)

# Merge browsing data with demographics data
df = gdf.merge(dem_df, how = 'left', on = 'machine_id')

### Total time spent (total number of visits) on pornographic domains

Given the potential skew in these numbers, we also show quartiles.

In [21]:
df.head()

,machine_id,total_time,total_visits_porn_prob_shalla_min_fn_fp,total_time_porn_prob_shalla_reduce_fp,total_visits,total_time_porn_prob_shalla_reduce_fn,total_time_porn_prob_shalla_min_fn_fp,total_visits_porn_prob_shalla_reduce_fp,total_time_porn_prob_toulouse_min_fn_fp,total_visits_porn_prob_toulouse_min_fn_fp,total_visits_porn_prob_shalla_reduce_fn,hoh_most_education,hoh_oldest_age
0,62,4245,265.0,1157.0,615,1162.0,1157.0,265.0,1157.0,265.0,270.0,Some college but no degree,45-49
1,2715,9419,145.0,416.0,920,488.0,417.0,144.0,417.0,145.0,153.0,Missing,25-29
2,3086,421,0.0,0.0,137,2.0,0.0,0.0,0.0,0.0,2.0,Missing,35-39
3,3325,21892,81.0,118.0,3302,415.0,173.0,62.0,184.0,86.0,206.0,High school diploma or equivalent,65 and over
4,3939,25430,73.0,203.0,4779,446.0,211.0,65.0,232.0,79.0,181.0,High school diploma or equivalent,45-49


We select the columns that we intend to show.

In [22]:
sel_cols = pd.IndexSlice[:, ['mean', '25%', '50%', '75%']]
grp_visits = ['total_visits_porn_prob_shalla_reduce_fn', 'total_visits_porn_prob_shalla_reduce_fp', 'total_visits_porn_prob_shalla_min_fn_fp', 'total_visits_porn_prob_toulouse_min_fn_fp']
grp_time = ['total_time_porn_prob_shalla_reduce_fn', 'total_time_porn_prob_shalla_reduce_fp', 'total_time_porn_prob_shalla_min_fn_fp', 'total_time_porn_prob_toulouse_min_fn_fp']

### Average Number of Visits to Pornographic Sites by Age

Given the skew, we focus our discussion on the medians. A consistent pattern emerges across all four versions of our measure: 18--20 visit the pornographic domains the most often but after that, there is a sharp decline and then a modest upward trend peaking at 40--44 after which the average number of visits roughly monotonically decline. You see the same rough pattern in the average time spent.

Perhaps yet more importantly, it seems the average number of visits are pretty low. We concur. And that means that the absolute size of the differences is pretty small too even though the relative size may look big. The more serious concern is about the data. We don't have a lot to say about it.

In [23]:
df.groupby(['hoh_oldest_age'])[grp_visits].describe().round(1).loc[:, sel_cols].astype('int')

total_visits_porn_prob_shalla_reduce_fn               \
                                                  mean 25% 50%  75%   
hoh_oldest_age                                                        
18-20                                              397  14  93  341   
21-24                                              309  11  65  245   
25-29                                              331  15  68  248   
30-34                                              324  17  72  247   
35-39                                              356  17  73  256   
40-44                                              345  19  83  283   
45-49                                              341  15  74  285   
50-54                                              352  13  62  242   
55-59                                              314  12  57  207   
60-64                                              246  10  47  184   
65 and over                                        244  10  47  183   

               total_visits_porn_prob_shalla_reduce_fp               \
                                                  mean 25% 50%  75%   
hoh_oldest_age                                                        
18-20                                              345   7  55  256   
21-24                                              266   4  34  172   
25-29                                              280   5  30  160   
30-34                                              270   5  30  152   
35-39                                              294   5  31  151   
40-44                                              282   6  38  176   
45-49                                              282   5  32  177   
50-54                                              293   4  24  140   
55-59                                              253   3  19  109   
60-64                                              187   2  14   80   
65 and over                                        183   2  13   80   

               total_visits_porn_prob_shalla_min_fn_fp               \
                                                  mean 25% 50%  75%   
hoh_oldest_age                                                        
18-20                                              356   8  61  268   
21-24                                              274   5  40  182   
25-29                                              288   6  36  175   
30-34                                              278   7  37  163   
35-39                                              304   7  37  168   
40-44                                              292   8  44  192   
45-49                                              291   6  38  189   
50-54                                              302   5  30  155   
55-59                                              261   5  24  122   
60-64                                              195   3  19   90   
65 and over                                        191   3  17   91   

               total_visits_porn_prob_toulouse_min_fn_fp               
                                                    mean 25% 50%  75%  
hoh_oldest_age                                                         
18-20                                                379  10  76  317  
21-24                                                294   8  52  227  
25-29                                                315   9  54  220  
30-34                                                306  11  55  219  
35-39                                                335  11  57  232  
40-44                                                324  13  67  255  
45-49                                                321  10  56  254  
50-54                                                331   7  47  209  
55-59                                                292   7  40  178  
60-64                                                224   5  32  151  
65 and over                                          221   6  33  149

### Average Time Spent on Pornographic Sites by Age

In [24]:
df.groupby(['hoh_oldest_age'])[grp_time].describe().round(1).loc[:, sel_cols].astype('int')

total_time_porn_prob_shalla_reduce_fn                 \
                                                mean 25%  50%   75%   
hoh_oldest_age                                                        
18-20                                           2238  48  399  1787   
21-24                                           1599  38  261  1219   
25-29                                           1735  53  272  1206   
30-34                                           1795  62  305  1213   
35-39                                           2031  64  293  1181   
40-44                                           1994  72  341  1345   
45-49                                           1915  55  303  1363   
50-54                                           2054  43  252  1145   
55-59                                           1770  42  222   972   
60-64                                           1472  33  194   826   
65 and over                                     1382  29  176   777   

               total_time_porn_prob_shalla_reduce_fp                 \
                                                mean 25%  50%   75%   
hoh_oldest_age                                                        
18-20                                           1902  19  230  1349   
21-24                                           1299   9  132   889   
25-29                                           1409  13  124   847   
30-34                                           1452  14  129   800   
35-39                                           1712  13  127   773   
40-44                                           1596  16  154   911   
45-49                                           1599  12  126   902   
50-54                                           1650   7   96   726   
55-59                                           1445   6   68   536   
60-64                                           1132   3   50   399   
65 and over                                     1062   3   44   373   

               total_time_porn_prob_shalla_min_fn_fp                 \
                                                mean 25%  50%   75%   
hoh_oldest_age                                                        
18-20                                           1991  23  253  1497   
21-24                                           1412  13  153   946   
25-29                                           1468  17  143   908   
30-34                                           1536  19  155   883   
35-39                                           1771  20  146   860   
40-44                                           1696  23  181  1002   
45-49                                           1664  17  149   982   
50-54                                           1725  11  116   800   
55-59                                           1502   9   84   609   
60-64                                           1196   5   65   472   
65 and over                                     1124   6   56   444   

               total_time_porn_prob_toulouse_min_fn_fp                 
                                                  mean 25%  50%   75%  
hoh_oldest_age                                                         
18-20                                             2087  29  304  1574  
21-24                                             1501  19  206  1046  
25-29                                             1590  29  203  1060  
30-34                                             1665  33  217  1022  
35-39                                             1890  32  212  1015  
40-44                                             1840  38  255  1163  
45-49                                             1783  28  215  1175  
50-54                                             1836  19  164   932  
55-59                                             1627  18  133   764  
60-64                                             1286  12  110   637  
65 and over                                       1224  12  105   587

### Average Number of Visits to Pornographic Sites by Education

As education levels increase, the average number of visits go down. Households where the most educated person in the household has a graduate degree visit pornographic sites less often and spent less time on them than households where the most educated person has less than a HS diploma.

In [25]:
df.groupby(['hoh_most_education'])[grp_visits].describe().round(1).loc[:, sel_cols].astype('int').reindex(edu.values())

total_visits_porn_prob_shalla_reduce_fn      \
                                                                     mean 25%   
hoh_most_education                                                              
Less than a high school diploma                                       355  19   
High school diploma or equivalent                                     316  15   
Some college but no degree                                            327  13   
Associate degree                                                      318  14   
Bachelor’s degree                                                     331  11   
Graduate degree                                                       288   9   
Missing                                                               331  16   

                                            \
                                  50%  75%   
hoh_most_education                           
Less than a high school diploma    92  346   
High school diploma or equivalent  67  245   
Some college but no degree         64  238   
Associate degree                   64  232   
Bachelor’s degree                  55  213   
Graduate degree                    47  194   
Missing                            76  272   

                                  total_visits_porn_prob_shalla_reduce_fp      \
                                                                     mean 25%   
hoh_most_education                                                              
Less than a high school diploma                                       294   7   
High school diploma or equivalent                                     256   5   
Some college but no degree                                            268   4   
Associate degree                                                      261   4   
Bachelor’s degree                                                     274   3   
Graduate degree                                                       234   2   
Missing                                                               271   5   

                                            \
                                  50%  75%   
hoh_most_education                           
Less than a high school diploma    41  213   
High school diploma or equivalent  28  144   
Some college but no degree         25  133   
Associate degree                   24  135   
Bachelor’s degree                  19  117   
Graduate degree                    15   93   
Missing                            33  167   

                                  total_visits_porn_prob_shalla_min_fn_fp      \
                                                                     mean 25%   
hoh_most_education                                                              
Less than a high school diploma                                       305   9   
High school diploma or equivalent                                     266   6   
Some college but no degree                                            277   5   
Associate degree                                                      269   5   
Bachelor’s degree                                                     282   4   
Graduate degree                                                       241   3   
Missing                                                               281   7   

                                            \
                                  50%  75%   
hoh_most_education                           
Less than a high school diploma    49  233   
High school diploma or equivalent  34  157   
Some college but no degree         30  147   
Associate degree                   30  146   
Bachelor’s degree                  24  129   
Graduate degree                    19  102   
Missing                            39  182   

                                  total_visits_porn_prob_toulouse_min_fn_fp  \
                                                                       mean   
hoh_most_education                                          

### Average Time Spent on Pornographic Sites by Education

In [26]:
df.groupby(['hoh_most_education'])[grp_time].describe().round(1).loc[:, sel_cols].astype('int').reindex(edu.values())

total_time_porn_prob_shalla_reduce_fn      \
                                                                   mean 25%   
hoh_most_education                                                            
Less than a high school diploma                                    2262  66   
High school diploma or equivalent                                  1877  54   
Some college but no degree                                         1819  46   
Associate degree                                                   1752  49   
Bachelor’s degree                                                  1832  39   
Graduate degree                                                    1706  28   
Missing                                                            1841  58   

                                              \
                                   50%   75%   
hoh_most_education                             
Less than a high school diploma    363  1553   
High school diploma or equivalent  277  1160   
Some college but no degree         251  1091   
Associate degree                   254  1065   
Bachelor’s degree                  222   963   
Graduate degree                    176   856   
Missing                            305  1304   

                                  total_time_porn_prob_shalla_reduce_fp      \
                                                                   mean 25%   
hoh_most_education                                                            
Less than a high school diploma                                    1862  18   
High school diploma or equivalent                                  1486  10   
Some college but no degree                                         1503   8   
Associate degree                                                   1446   8   
Bachelor’s degree                                                  1488   5   
Graduate degree                                                    1396   3   
Missing                                                            1488  13   

                                              \
                                   50%   75%   
hoh_most_education                             
Less than a high school diploma    178  1127   
High school diploma or equivalent  110   754   
Some college but no degree          95   676   
Associate degree                    93   690   
Bachelor’s degree                   68   567   
Graduate degree                     45   434   
Missing                            132   856   

                                  total_time_porn_prob_shalla_min_fn_fp      \
                                                                   mean 25%   
hoh_most_education                                                            
Less than a high school diploma                                    1930  25   
High school diploma or equivalent                                  1576  15   
Some college but no degree                                         1560  13   
Associate degree                                                   1514  12   
Bachelor’s degree                                                  1563   9   
Graduate degree                                                    1465   5   
Missing                                                            1566  18   

                                              \
                                   50%   75%   
hoh_most_education                             
Less than a high school diploma    209  1218   
High school diploma or equivalent  132   830   
Some college but no degree         116   759   
Associate degree                   117   756   
Bachelor’s degree                   87   636   
Graduate degree                     62   505   
Missing                            155   940   

                                  total_time_porn_prob_toulouse_min_fn_fp      \
                                                                     mean 25%   
hoh_most_education                                      

### Proportion of Time Spent on (Proportion of Visits to) Pornographic Domains

Do we see the patterns because it just captures that certain people spend more time online? To check that we look at proportions.

The data are clear---as people get older, they generally spend a smaller proportion of time on pornographic websites with perceptible drop-offs after 50--54. Splitting by education also shows that the declining trend is a result of people in households where education level is higher spending less time on pornographic domains.

In [27]:
grp_prop_visits = []
for g in grp_visits:
    df['prop_' + g] = df[g]/df['total_visits'] 
    grp_prop_visits.append('prop_'  + g)

grp_prop_time = []
for g in grp_time:
    df['prop_' + g] = df[g]/df['total_time'] 
    grp_prop_time.append('prop_'  + g)    

#### By Age

In [28]:
df.groupby(['hoh_oldest_age'])[grp_prop_visits].mean().round(3)

,prop_total_visits_porn_prob_shalla_reduce_fn,prop_total_visits_porn_prob_shalla_reduce_fp,prop_total_visits_porn_prob_shalla_min_fn_fp,prop_total_visits_porn_prob_toulouse_min_fn_fp
hoh_oldest_age,,,,
18-20,0.105,0.089,0.092,0.100
21-24,0.099,0.083,0.086,0.094
25-29,0.097,0.078,0.081,0.091
30-34,0.091,0.072,0.075,0.085
35-39,0.089,0.069,0.071,0.083
40-44,0.090,0.070,0.073,0.084
45-49,0.091,0.071,0.074,0.085
50-54,0.087,0.068,0.070,0.081
55-59,0.079,0.059,0.062,0.073


In [29]:
df.groupby(['hoh_oldest_age'])[grp_prop_time].mean().round(3)

,prop_total_time_porn_prob_shalla_reduce_fn,prop_total_time_porn_prob_shalla_reduce_fp,prop_total_time_porn_prob_shalla_min_fn_fp,prop_total_time_porn_prob_toulouse_min_fn_fp
hoh_oldest_age,,,,
18-20,0.061,0.052,0.054,0.057
21-24,0.060,0.051,0.053,0.056
25-29,0.060,0.051,0.052,0.055
30-34,0.056,0.047,0.048,0.052
35-39,0.053,0.044,0.046,0.049
40-44,0.054,0.044,0.046,0.050
45-49,0.055,0.046,0.048,0.051
50-54,0.055,0.045,0.047,0.050
55-59,0.050,0.041,0.042,0.046


#### By Education

In [30]:
df.groupby(['hoh_most_education'])[grp_prop_visits].mean().round(3).reindex(edu.values())

,prop_total_visits_porn_prob_shalla_reduce_fn,prop_total_visits_porn_prob_shalla_reduce_fp,prop_total_visits_porn_prob_shalla_min_fn_fp,prop_total_visits_porn_prob_toulouse_min_fn_fp
hoh_most_education,,,,
Less than a high school diploma,0.099,0.080,0.083,0.093
High school diploma or equivalent,0.087,0.068,0.070,0.081
Some college but no degree,0.085,0.066,0.068,0.079
Associate degree,0.085,0.066,0.068,0.079
Bachelor’s degree,0.082,0.063,0.065,0.075
Graduate degree,0.076,0.057,0.059,0.069
Missing,0.091,0.071,0.073,0.084


In [31]:
df.groupby(['hoh_most_education'])[grp_prop_time].mean().round(3).reindex(edu.values())

,prop_total_time_porn_prob_shalla_reduce_fn,prop_total_time_porn_prob_shalla_reduce_fp,prop_total_time_porn_prob_shalla_min_fn_fp,prop_total_time_porn_prob_toulouse_min_fn_fp
hoh_most_education,,,,
Less than a high school diploma,0.058,0.050,0.051,0.055
High school diploma or equivalent,0.055,0.045,0.047,0.050
Some college but no degree,0.052,0.043,0.044,0.047
Associate degree,0.052,0.043,0.045,0.048
Bachelor’s degree,0.052,0.043,0.044,0.048
Graduate degree,0.049,0.040,0.042,0.045
Missing,0.055,0.045,0.046,0.050
